In [ ]:
import numpy as np
import tensorflow as tf

from keras import backend as K
from scipy.optimize import fmin_l_bfgs_b

from keras.applications import vgg16, vgg19
from keras.preprocessing.image import load_img

In [ ]:
# https://github.com/kevinzakka/style-transfer/blob/master/neural_styler.py

# image processing

In [ ]:
# util function to open, resize and format pictures into appropriate tensors
def preprocess_image(image_path, desired_dims):
    img = load_img(image_path, target_size=desired_dims)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
return img



In [ ]:
# util function to convert a tensor into a valid image	
def deprocess_image(x, img_nrows, img_ncols):
    if K.image_dim_ordering() == 'th':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
return x

# loss functions

In [ ]:
def feature_reconstruction_loss(base, output):
    """
    Feature reconstruction loss function. Encourages the 
    output img to be perceptually similar to the base image.
    """
return K.sum(K.square(output - base))


In [ ]:
def gram_matrix(x):
    """
    Computes the outer-product of the input tensor x.
    Input
    -----
    - x: input tensor of shape (C x H x W)
    
    Returns
    -------
    - x . x^T
    Note that this can be computed efficiently if x is reshaped
    as a tensor of shape (C x H*W).
    """
    # assert K.ndim(x) == 3
    if K.image_dim_ordering() == 'th':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
return K.dot(features, K.transpose(features))

In [ ]:
def style_reconstruction_loss(base, output, img_nrows, img_ncols):
    """
    Style reconstruction loss. Encourage the output img 
    to have same stylistic features as style image. Does not
    preserve spatial structure however.
    """
    H, W, C = img_nrows, img_ncols, 3
    gram_base = gram_matrix(base)
    gram_output = gram_matrix(output)
    factor = 1.0 / float((2*C*H*W)**2)
    out = factor * K.sum(K.square(gram_output - gram_base))
return out



In [ ]:
def total_variation_loss(x, img_nrows, img_ncols):
    """
    Total variational loss. Encourages spatial smoothness 
    in the output image.
    """
    H, W = img_nrows, img_ncols
    if K.image_dim_ordering() == 'th':
        a = K.square(x[:, :, :H-1, :W-1] - x[:, :, 1:, :W-1])
        b = K.square(x[:, :, :H-1, :W-1] - x[:, :, :H-1, 1:])
    else:
        a = K.square(x[:, :H-1, :W-1, :] - x[:, 1:, :W-1, :])
        b = K.square(x[:, :H-1, :W-1, :] - x[:, :H-1, 1:, :])

return K.sum(K.pow(a + b, 1.25))


# Model

In [ ]:
def grads(self, x):
    # reshape
    if K.image_dim_ordering() == 'th':
        x = x.reshape((1, 3, self.img_nrows, self.img_ncols))
    else:
        x = x.reshape((1, self.img_nrows, self.img_ncols, 3))

    outs = self.loss_and_grads([x])

    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
return grad_values

In [ ]:
print('\nInitializing Style Transfer model...')

# store paths
base_img_path = ./base_img/
style_img_path = ./style_img/
output_img_path = ./output_img/

# define constant
content_weight = 7.5e0
style_weight = 1e2
tv_weight = 2e2
iterations = 100
content_layer = 'block4_conv2'
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1',
                'block5_conv1']

In [ ]:

print('\n\tResizing images...')
#self.width = output_width
width, height = load_img(self.base_img_path).size
new_dims = (height, width)

# store shapes for future use
#img_nrows = height
#img_ncols = width

# resize content and style images to this desired shape
content_img = K.variable(preprocess_image(base_img_path, new_dims))
style_img = K.variable(preprocess_image(style_img_path, new_dims))

# and also create output placeholder with desired shape
if K.image_dim_ordering() == 'th':
    output_img = K.placeholder((1, 3, new_dims[0], new_dims[1]))
else:
    output_img = K.placeholder((1, new_dims[0], new_dims[1], 3))

    
# sanity check on dimensions
print("\tSize of content image is: {}".format(K.int_shape(content_img)))
print("\tSize of style image is: {}".format(K.int_shape(style_img)))
print("\tSize of output image is: {}".format(K.int_shape(output_img)))

In [ ]:
# create a tensor with the 3 images
input_img = K.concatenate([content_img, style_img, output_img], axis=0)


In [ ]:
# initialize the vgg16 /vgg19 model

covnet = vgg16
#covnet = vgg19

print('\tLoading {} model'.format(convnet.upper()))

if convnet == 'vgg16':
    model = vgg16.VGG16(input_tensor=input_img,
                        weights='imagenet',
                        include_top=False)
else:
    model = vgg19.VGG19(input_tensor=input_img,
                        weights='imagenet',
                        include_top=False)

In [1]:
print('\tComputing losses...')
# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

	Computing losses...


NameError: name 'model' is not defined

In [ ]:
# extract features only from the content layer
content_features = outputs_dict[content_layer]

In [ ]:
# extract the activations of the base image and the output image
base_image_features = content_features[0, :, :, :] 	# 0 corresponds to base
combination_features = content_features[2, :, :, :] # 2 coresponds to output

In [ ]:
# calculate the feature reconstruction loss
content_loss = content_weight * feature_reconstruction_loss(base_image_features, combination_features)
print('\n\t content_loss: ' content_loss)

In [ ]:

# for each style layer compute style loss
# total style loss is then weighted sum of thoscontent_weighte losses
temp_style_loss = K.variable(0.0)
weight = 1.0 / float(len(style_layers))

for layer in style_layers:
    # extract features of given layer
    style_features = outputs_dict[layer]
    # from those features, extract style and output activations
    style_image_features = style_features[1, :, :, :]
    output_style_features = style_features[2, :, :, :]
    temp_style_loss += weight * \
                        style_reconstruction_loss(style_image_features, 
                                                  output_style_features,
                                                  img_nrows, 
                                                  img_ncols)
style_loss = style_weight * temp_style_loss
print('\t style_loss: ' style_loss)






In [ ]:
# compute total variational loss
tv_loss = tv_weight * total_variation_loss(output_img, 
                                           img_nrows, 
                                           img_ncols)
print('\t tv_loss: ' tv_loss)

In [ ]:
# composite loss
total_loss = content_loss + style_loss + tv_loss
print('\t total_loss: ' total_loss)


In [ ]:
# compute gradients of output img with respect to loss
print('\tComputing gradients...')
grads = K.gradients(total_loss, output_img)

outputs = [total_loss]

if type(grads) in {list, tuple}:
    outputs += grads
else:
    outputs.append(grads)

loss_and_grads = K.function([output_img], outputs)
